In [20]:
import pandas as pd
df_1997= pd.read_csv('data/processed/1997.csv',encoding="ISO-8859-1")

In [43]:
# data processing to only view california real estate data
df_sale_counts_by_zip = pd.read_csv('data/raw/Sale_Counts_Zip.csv', encoding="ISO-8859-1")
df_median_all_home_price_by_zip = pd.read_csv('data/raw/Zip_MedianListingPrice_AllHomes.csv',  encoding="ISO-8859-1")
df_all_home_by_zip = pd.read_csv('data/raw/Zip_Zhvi_AllHomes.csv',  encoding="ISO-8859-1")
df_single_family_residence_by_zip = pd.read_csv('ction/data/raw/Zip_Zhvi_SingleFamilyResidence.csv',  encoding="ISO-8859-1")

df_sale_counts_by_zip_california = df_sale_counts_by_zip.loc[df_sale_counts_by_zip['StateName']=='California']
df_all_home_by_zip_california = df_all_home_by_zip.loc[df_all_home_by_zip['State']=='CA']
df_single_family_residence_by_zip_california = df_single_family_residence_by_zip.loc[df_single_family_residence_by_zip['State']=='CA']
df_median_all_home_price_by_zip_california = df_median_all_home_price_by_zip[df_median_all_home_price_by_zip['State']=='CA']

In [40]:
# data processing to load all IPO Data between 1997 and present data. This data has been scraped using code from src/web_scrapers.py
df_full_ipo_date_97_to_19 = pd.read_csv('/Users/aaron/Development/SF-home-price-prediction/data/processed/1997-04_2019_full_ipo_data.csv',encoding="ISO-8859-1")

In [44]:
df_sale_counts_by_zip.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,seasAdj
0,61639.0,10025,New York,1,NaN,NaN,NaN,NaN,NaN,NaN,...,83.0,62.0,44.0,63.0,49.0,75.0,31.0,47.0,54.0,0
1,84654.0,60657,Illinois,2,NaN,NaN,NaN,NaN,NaN,NaN,...,189.0,161.0,136.0,110.0,99.0,94.0,NaN,NaN,NaN,0
2,61637.0,10023,New York,3,NaN,NaN,NaN,NaN,NaN,NaN,...,111.0,59.0,92.0,61.0,75.0,77.0,42.0,62.0,45.0,0
3,91982.0,77494,Texas,4,55.0,71.0,83.0,93.0,116.0,86.0,...,246.0,147.0,202.0,149.0,167.0,84.0,85.0,30.0,NaN,0
4,84616.0,60614,Illinois,5,NaN,NaN,NaN,NaN,NaN,NaN,...,196.0,141.0,132.0,154.0,83.0,74.0,90.0,145.0,198.0,0
